# Torre de Hanoi
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

El rompecabezas comienza con los discos apilados en una varilla en orden decreciente de tamaño, con el disco más pequeño en la parte superior, formando una especie de figura cónica.

El objetivo del juego es mover toda la pila de discos a una de las otras varillas, cumpliendo con las siguientes reglas:

1. Solo se puede mover un disco a la vez.
2. Cada movimiento consiste en tomar el disco superior de una de las pilas y colocarlo sobre otra pila o sobre una varilla vacía.
3. Ningún disco puede colocarse sobre uno que sea más pequeño.

## Resolviendo este problema usando IA

Este es un problema clásico ideal para aplicar métodos de búsqueda. Podemos construir un agente inteligente que resuelva el problema automáticamente.

Este agente percibe cuántos discos hay y en qué orden se encuentran en cada varilla. Además, puede tomar cualquier disco que esté en la parte superior de una pila y moverlo a otra varilla, siempre que la jugada sea válida.

### Definición del problema

Para resolverlo, necesitamos definir correctamente el espacio de estados, el estado inicial, el estado objetivo y las reglas de transición.

#### Espacio de estados:

Para 5 discos, existen $3^5 = 243$ posibles configuraciones (ya que cada disco puede estar en una de las tres varillas).

![estados_hanoi](./img/state_hanoi1.png)

#### Estado inicial

En nuestro caso, partimos con todos los discos apilados en la **varilla izquierda**, ordenados de mayor a menor de abajo hacia arriba.

![estados_hanoi_initial](./img/state_hanoi2.png)

#### Estado objetivo

Para simplificar, definimos un único estado objetivo: todos los discos deben terminar ordenados de la misma forma en la **varilla derecha**.

![estados_hanoi_goal](./img/state_hanoi3.png)

----
### Implementación

Vamos a ver una implementación de las diferentes estructuras necesarias para resolver el problema de la Torre de Hanoi utilizando algoritmos de búsqueda, como vimos en el video de resolución de problemas. Para esto, nos basamos en el código del libro *Artificial Intelligence: A Modern Approach de Russell y Norvig*, disponible en GitHub:

- 📚 [Repositorio AIMA](https://github.com/aimacode)
- 🐍 [Versión en Python](https://github.com/aimacode/aima-python)

> 🧠 **Nota 1**: Si querés desarrollar una versión en otro lenguaje, en el repositorio hay implementaciones en varios lenguajes.

> ⚠️ **Nota 2**: Para profundizar en la implementación y cómo está estructurado el código, recomendamos revisar el archivo `hanoi_states.py` ubicado en `lib`, el cual está ampliamente comentado.

### Representación del estado

Cualquier estado del problema puede representarse mediante la clase `StatesHanoi`:

In [1]:
from aima_libs.hanoi_states import StatesHanoi

Para representar la ubicación de los discos, usamos tres listas (una por varilla), y asignamos a cada disco un número del 1 al 5. Por ejemplo, si queremos representar el siguiente estado:

![estados_hanoi_goal](./img/state_hanoi0.png)

In [2]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]

In [3]:
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Podemos imprimir el estado directamente:

In [4]:
print(state)

HanoiState:  | 5 3 1 | 4 2


#### Métodos disponibles

Obtener el disco superior de una varilla:

In [ ]:
disk = state.get_last_disk_rod(number_rod=1)
print(f"El ultimo disco de la varilla del centro es {disk}")

El ultimo disco de la varilla del centro es 1


Verificar si es válido colocar un disco en una varilla:

In [ ]:
disk = 1
print("Podemos poner el disco 1 en la varilla derecha?")
if state.check_valid_disk_in_rod(number_rod=2, disk=disk):
    print("Si, es posible poner el disco 1 en la varilla de derecha")

Podemos poner el disco 1 en la varilla derecha?
Si, es posible poner el disco 1 en la varilla de derecha


Aplicar un movimiento válido y modificar el estado:

In [ ]:
state.put_disk_in_rod(number_rod=2, disk=disk)
print(f"El nuevo estado es: {state}")

El nuevo estado es: HanoiState:  | 5 3 | 4 2 1


Este movimiento genera el siguiente estado:

![estados_hanoi_goal](./img/state_hanoi4.png)

#### Detalles de la clase `StatesHanoi`

**Atributos**:
* `rods`: Lista de listas que contiene los discos en cada varilla.
* `number_of_disks`: Cantidad total de discos (se puede usar con cualquier número).
* `number_of_pegs`: Número de varillas (en este caso, siempre 3).
* `accumulated_cost`: Costo acumulado del camino hasta el estado actual (útil para algoritmos como A*).

**Métodos:**
* `accumulate_cost()`: Suma un valor al costo acumulado.
* `get_state()`: Devuelve la representación en listas del estado.
* `get_state_dict()`: Devuelve el estado en formato de diccionario.
* `get_accumulated_cost()`: Devuelve el costo acumulado.
* `check_valid_disk_in_rod()`: Verifica si es válido colocar un disco en cierta varilla.
* `get_last_disk_rod()`: Devuelve el último disco (el que está arriba) de una varilla.
* `put_disk_in_rod()`: Aplica un movimiento colocando el disco en la varilla.

**Operaciones adicionales:**

La clase también permite realizar operaciones útiles directamente en Python:
* Comparar estados: `estado1 == estado2`.
* Comparar costos: `estado1 > estado2` (si el costo acumulado de `estado1` es mayor)
* Representación en texto: al hacer `print(estado)` se imprime el estado de forma legible.
* Hashing: permite usar los estados como claves en sets o diccionarios: `hash(estado)`

---
### Acciones

Además de definir los estados, también debemos contemplar las acciones que permiten pasar de un estado a otro. En este caso, mover un disco de una varilla a otra. Para eso utilizamos la clase `ActionHanoi`:

In [8]:
from aima_libs.hanoi_states import ActionHanoi

Veamos cómo aplicar una acción que transforma este estado:

![estados_hanoi_goal](./img/state_hanoi0.png)

En este otro:

![estados_hanoi_goal](./img/state_hanoi4.png)

Es decir, queremos mover el disco 1 desde la **varilla del medio** hacia la **varilla derecha**.

In [9]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Primero creamos la acción. En este punto, la acción aún no ha sido ejecutada:

In [10]:
action_example = ActionHanoi(disk=1, rod_input=1, rod_out=2)
print(action_example)

Move disk 1 from 2 to 3


Ahora sí, aplicamos la acción al estado `state`. Esto genera un nuevo estado:

In [11]:
new_state = action_example.execute(state_hanoi=state)

print(new_state)

HanoiState:  | 5 3 | 4 2 1


Notamos que el costo acumulado empieza a cobrar sentido: *mover un disco tiene un costo igual a 1*, por lo que el nuevo estado reflejará ese costo:

In [12]:
print(f"El costo acumulado del nuevo estado es {new_state.accumulated_cost}")

El costo acumulado del nuevo estado es 1.0


#### Detalles de la clase `ActionHanoi`

**Atributos:**
* `disk`: Disco que se desea mover.
* `rod_input`: Varilla desde la cual se retira el disco.
* `rod_out`: Varilla a la cual se mueve el disco.
* `action`: Cadena de caracteres descriptiva de la acción.
* `action_dict`: Representación de la acción en formato diccionario.
* `cost`: Costo asociado a la acción (1 por defecto, o 0 si la acción no implica un movimiento real).

**Métodos:**
* `execute(state_hanoi)`: Aplica la acción sobre un estado dado. Devuelve un nuevo estado con el disco movido y el costo acumulado actualizado.

**Operaciones adicionales implementadas:**

La clase `ActionHanoi` también permite operar naturalmente dentro del entorno de Python:

* Cuenta con una representación en string (`__str__`), por lo que al hacer `print(action)` se obtiene una descripción clara de la acción.
* Permite convertir fácilmente la acción en un diccionario (`action_dict`), lo cual es útil al momento de exportar la solución, por ejemplo, al simulador visual de PyGame.

---
### Problema de Hanoi

Por último, podemos implementar el problema completo, que contemple tanto el estado inicial como el estado final, y la posibilidad de realizar movimientos de un estado a otro. De esta manera, podremos movernos a través del grafo de estados. Nuestra implementación se llama `ProblemHanoi`.

In [13]:
from aima_libs.hanoi_states import ProblemHanoi

En este problema, definimos primero el estado inicial desde donde comenzamos:

![estados_hanoi_initial](./img/state_hanoi2.png)

In [14]:
varilla_izquierda = [5, 4, 3, 2, 1]
varilla_medio = []
varilla_derecha = []

initial_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Y el estado objetivo al que queremos llegar:

![estados_hanoi_goal](./img/state_hanoi3.png)

In [15]:
varilla_izquierda = []
varilla_medio = []
varilla_derecha = [5, 4, 3, 2, 1]
goal_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Con estos estados, definimos el problema de la Torre de Hanoi:

In [16]:
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

#### Detalles de la clase `ProblemHanoi`

**Atributos:**
Esta clase tiene dos atributos principales:
* `initial`: Es el estado inicial.
* `goal`: Es el estado objetivo.

**Métodos:**
* `actions()`: Devuelve todas las acciones posibles que se pueden ejecutar desde un estado dado.
* `result()`: Calcula el nuevo estado después de aplicar una acción.
* `path_cost()`: Calcula el costo del camino de un estado a otro.
* `goal_test()`: Verifica si un estado particular es el estado objetivo.

#### Ejemplo de uso

Veamos cómo podemos obtener todas las acciones que se pueden aplicar desde un estado dado:

In [17]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

In [18]:
lista_acciones = problem.actions(state)
for action in lista_acciones:
    print(action)

Move disk 1 from 2 to 1
Move disk 1 from 2 to 3
Move disk 2 from 3 to 1


Ahora, aplicamos una de las acciones que nos devuelve la lista:

In [ ]:
# Aplicamos la acción de mover el disco 1 de la varilla 2 a la varilla 3
new_state = problem.result(state=state, action=lista_acciones[1])

print(new_state)

HanoiState:  | 5 3 | 4 2 1


Esto nos lleva de este estado:

![estados_hanoi_goal](./img/state_hanoi0.png)

A este estado:

![estados_hanoi_goal](./img/state_hanoi4.png)

Pero ahora, dentro del contexto del **Problema**.

Acumulemos el costo que venimos acumulando hasta el nuevo estado:

In [20]:
print(f"El costo acumulado según el problema del nuevo estado es {problem.path_cost(c=1, state1=state, action=lista_acciones[1], state2=new_state)}")

El costo acumulado según el problema del nuevo estado es 1.0


Aplicamos otra acción al nuevo estado:

In [21]:
lista_acciones = problem.actions(new_state)
for action in lista_acciones:
    print(action)

Move disk 3 from 2 to 1
Move disk 1 from 3 to 1
Move disk 1 from 3 to 2


In [ ]:
# Aplicamos la acción de mover el disco 3 de la varilla 2 a la varilla 1
new_state_2 = problem.result(state=new_state, action=lista_acciones[0])

print(new_state_2)

HanoiState: 3 | 5 | 4 2 1


Ahora, el costo acumulado es 2, ya que hemos pasado por dos estados para llegar a este nuevo estado:

In [23]:
print(f"El costo acumulado según el problema del nuevo estado es {problem.path_cost(c=1, state1=new_state, action=lista_acciones[0], state2=new_state_2)}")

El costo acumulado según el problema del nuevo estado es 2.0


**Verificando si un estado es la solución:**

Finalmente, podemos verificar si un estado dado es la solución al problema:

In [24]:
if not problem.goal_test(state=new_state_2):
    print(f"{new_state_2} no es la solución final {goal_state}")

HanoiState: 3 | 5 | 4 2 1 no es la solución final HanoiState:  |  | 5 4 3 2 1


--- 
### Grafo de Estados de Hanoi

Con esta implementación, ya podemos generar el grafo de estados de la Torre de Hanoi, que nos muestra todos los posibles estados alcanzables desde el estado inicial hasta el objetivo.

![grafo_de_hanoi](./img/state_hanoi_graph.png)